In [1]:
import os
import tensorflow as tf
from tensorflow.python.lib.io import file_io
import numpy as np
from collections import defaultdict
from random import shuffle
from scipy import signal
from scipy.io import wavfile
from scipy import interpolate

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

### Set global variables

In [3]:
TFR_TRAIN = 'train.tfrecord'
TFR_VALID = 'valid.tfrecord'
TFR_TEST = 'test.tfrecord'
BUCKET = 'gs://robolab/'

NUM_CLASSES = 2

IMG_HEIGHT = 80 # four concatenated cropped specs = 4 * 20
IMG_WIDTH = 71

### TF-Parsers

In [4]:
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [5]:
def parser(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.string)})

    image = tf.decode_raw(features['image_raw'], tf.float32)
    image.set_shape([IMG_HEIGHT * IMG_WIDTH])

    label = tf.decode_raw(features['label'], tf.int32)
    label.set_shape([1])

    return image, label

In [6]:
def test_parser(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={'image_raw': tf.FixedLenFeature([], tf.string)})

    image = tf.decode_raw(features['image_raw'], tf.float32)
    image.set_shape([IMG_HEIGHT * IMG_WIDTH])

    return image

### Input functions

In [7]:
def train_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TRAIN)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

In [8]:
def eval_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TRAIN)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

In [9]:
def valid_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_VALID)

    # map parser over dataset samples
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features, labels = iterator.get_next()

    return features, labels

In [10]:
def predict_input_fn():

    # get dataset from tf_record
    dataset = tf.data.TFRecordDataset(BUCKET + TFR_TEST)

    # map parser over dataset samples
    dataset = dataset.map(test_parser)
    dataset = dataset.batch(1)
    dataset = dataset.repeat(1)
    iterator = dataset.make_one_shot_iterator()

    features = iterator.get_next()

    return features

### CNN model

In [11]:
def cnn_model_fn(features, labels, mode):
    
    input_layer = tf.reshape(features, [-1, IMG_HEIGHT, IMG_WIDTH], name='input_bytes')
    input_layer = tf.expand_dims(input_layer, axis=3)

    conv_layer_1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=8,
        kernel_size=[2, 2],
        padding='same',
        activation=tf.nn.relu)

    pool_layer_1 = tf.layers.max_pooling2d(
        inputs=conv_layer_1,
        pool_size=[2, 2],
        strides=2,
        padding='same')

    conv_layer_2 = tf.layers.conv2d(
        inputs=pool_layer_1,
        filters=32,
        kernel_size=[2, 2],
        padding='same',
        activation=tf.nn.relu)

    pool_layer_2 = tf.layers.max_pooling2d(
        inputs=conv_layer_2,
        pool_size=[2, 2],
        strides=2,
        padding='same')

    reshape_layer = tf.layers.flatten(pool_layer_2)

    dense_layer = tf.layers.dense(
        inputs=reshape_layer,
        units=256,
        activation=tf.nn.relu)
    
    is_train = False

    if mode == tf.estimator.ModeKeys.TRAIN:
        is_train = True

    dropout_layer = tf.layers.dropout(
        inputs=dense_layer,
        rate=0.2,
        training=is_train)

    logits_layer = tf.layers.dense(
        inputs=dropout_layer,
        units=NUM_CLASSES)

    predictions = {
        'classes':tf.argmax(logits_layer, axis=1),
        'probabilities':tf.nn.softmax(logits_layer, axis=1)}

    serving_output = tf.estimator.export.ClassificationOutput(scores=predictions['probabilities'])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions,
                                          export_outputs={'x':serving_output})

    loss = tf.losses.sparse_softmax_cross_entropy(
        labels=labels,
        logits=logits_layer)

    accuracy = tf.metrics.accuracy(
        labels=labels,
        predictions=tf.argmax(logits_layer, axis=1),
        name='accu_op')

    if mode == tf.estimator.ModeKeys.TRAIN:
        train_optimizer = tf.train.AdamOptimizer(learning_rate=LR).minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_optimizer)

    # mode = EVAL
    eval_metric_ops = {'accuracy':accuracy}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### Configure estimator

In [12]:
OUTDIR = BUCKET + 'output'

cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=OUTDIR)

### Create serving_input_receiver_fn for serving model

In [2]:
def serving_input_receiver_fn():
    
    serialized_example = tf.placeholder(dtype=tf.string,
                                       shape=[None],
                                       name='raw_bytes')
    
    receiver_tensors = {'examples': serialized_example}
    
    feats = tf.parse_example(
        serialized_example,
        {'image_raw': tf.FixedLenFeature([], tf.string)})
    
    fn = lambda x: tf.decode_raw(x, tf.float32)
    
    feats = tf.map_fn(fn, feats['image_raw'], dtype=tf.float32, name='parser')
    feats.set_shape([None, IMG_HEIGHT * IMG_WIDTH])
    
    return tf.estimator.export.TensorServingInputReceiver(feats, receiver_tensors)

In [42]:
cnn_classifier.export_saved_model('gs://robolab/saved_model', serving_input_receiver_fn)

Tensor("parser/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, 5680), dtype=float32)


b'gs://robolab/saved_model/1543691652'